In [1]:
import torch
from ConditionalImitationLearningNetwork import ImitationLearningNetwork_Training
from CarlaDatasetLoader import CarlaDataset
from utils import get_batch_mask
from torch.utils.data import DataLoader
from torchvision import transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ds = CarlaDataset("data\data.csv", for_training=True)
dl = DataLoader(dataset =ds, num_workers=1, batch_size=30, shuffle=True)
for index, (img, speed, target_vec, mask_vec) in enumerate(dl):
    img1 = np.asarray( img[1].permute(  1, 2, 0), dtype= np.float32) / 255.
    plt.imshow(img1 )
    plt.show()
    


KeyboardInterrupt: 

In [ ]:
# img, speed, commands_masks = torch.randn((20, 3, 88, 200)), torch.randn(20, 1), get_batch_mask(torch.randint(0, 3, (20,1)))
# model = ImitationLearningNetwork_Training()
# x = model(img, speed)